In [1]:
#| default_exp routes.group

In [2]:
#| exporti
import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de

In [3]:
# | export
def generate_body_create_group(group_name: str,
                               group_type: str = 'open',
                               description: str = '') -> dict:
    """ Generates the body to create group for content_v2_group API"""
    body = {"name": group_name, 
            "type": group_type,
            "description": description}

    return body

#### sample implementation of generate_body_create_group

In [4]:
generate_body_create_group(
    group_name='test_group_name', 
    group_type='open', 
    description='from jupyter')


{'name': 'test_group_name', 'type': 'open', 'description': 'from jupyter'}

In [5]:
#| export
async def search_groups_by_name(auth: dmda.DomoAuth,
                                search_name: str,
                                debug_api: bool = False, 
                                log_results: bool = False) -> rgd.ResponseGetData:
    url = f'https://{auth.domo_instance}.domo.com/api/content/v2/groups/grouplist?ascending=true&search={search_name}&sort=name '

    res = await gd.get_data(
        auth=auth,
        url=url,
        method='GET',
        debug_api=debug_api
    )
    return res

#### Sample implementation of search_groups_by_name

In [6]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community", 
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await search_groups_by_name(auth=token_auth, search_name='Test Group 2')
res

ResponseGetData(status=200, response=[{'name': 'Test Group 2', 'groupId': 1259653287, 'owners': [{'type': 'USER', 'id': '1893952720', 'displayName': 'Jae Wilson1'}, {'type': 'GROUP', 'id': '1259653287', 'displayName': 'Test Group 2'}], 'groupType': 'open', 'groupMembers': [{'type': 'USER', 'id': '2072616249'}], 'memberCount': 1, 'created': '2023-02-15 17:10:37.0', 'description': ''}, {'name': 'Admin Test', 'groupId': 1814479647, 'owners': [{'type': 'USER', 'id': '55874022', 'displayName': 'Grant Smith'}, {'type': 'GROUP', 'id': '822382906', 'displayName': 'Grant: Manage all groups'}], 'groupType': 'closed', 'groupMembers': [{'type': 'USER', 'id': '55874022'}], 'memberCount': 1, 'created': '2023-01-29 23:37:13.0', 'description': ''}, {'name': 'April 2023 Cohort', 'groupId': 926594289, 'owners': [{'type': 'GROUP', 'id': '822382906', 'displayName': 'Grant: Manage all groups'}, {'type': 'USER', 'id': '583215149', 'displayName': 'John Jarnagin'}, {'type': 'USER', 'id': '617808774', 'display

In [7]:
#| export
class CreateGroup_Error(de.DomoError):
    def __init__(self, status, message, domo_instance, function_name = "create_group"):
        super().__init__(function_name = function_name, status = status, message = message , domo_instance = domo_instance)

async def create_group(auth: dmda.DomoAuth,
                       group_name: str,
                       group_type: str = 'open',
                       description: str = '',
                       debug_api: bool = False,
                       session: httpx.AsyncClient = None
                       ) -> rgd.ResponseGetData:
    # body : {"name": "GROUP_NAME", "type": "open", "description": ""}

    body = generate_body_create_group(
        group_name=group_name, group_type=group_type, description=description)

    url = f'https://{auth.domo_instance}.domo.com/api/content/v2/groups/'

    res= await gd.get_data(
        auth=auth,
        url=url,
        method='POST',
        body=body,
        debug_api=debug_api,
        session = session
    )

    if not res.is_success:
        group_exists = await search_groups_by_name(auth=auth, search_name=group_name)
        if group_exists.is_success:
            raise CreateGroup_Error(
                status = res.status,
                message = f'{group_name} already exists. Choose a different group_name',
                domo_instance = auth.domo_instance,
                function_name='create_group'
            )

    if not res.is_success:
        raise CreateGroup_Error(
            status = res.status, 
            message = res.response,
            domo_instance = auth.domo_instance, 
            function_name="create_group")

    return res


#### Sample implementation of create_group

In [8]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community", 
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)
try:
    res = await create_group(auth=token_auth, group_name='Test Group 2', debug_api=False)
    res.response

except CreateGroup_Error as e:
    print(e)

create_group: Status 400 - Test Group 2 already exists. Choose a different group_name at domo-community


In [9]:
#| export
async def get_all_groups(auth: dmda.DomoAuth,
                         log_results: bool = False,
                         debug_api: bool = False,
                         session: httpx.AsyncClient = None) -> rgd.ResponseGetData:
    if debug_api:
        print(auth)

    url = f'https://{auth.domo_instance}.domo.com/api/content/v2/groups/grouplist'

    if debug_api:
        print(auth, url)

    if log_results:
        print(f'Getting groups from - {url}')

    res = await gd.get_data(url=url, method='GET', auth=auth, session=session)

    return res

#### Sample implementation of get_all_groups

In [10]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community", 
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await get_all_groups(auth=token_auth)
res_df = pd.DataFrame(res.response)
res_df[0:1]

,name,groupId,owners,groupType,groupMembers,memberCount,created,description
0,Admin Test,1814479647,"[{'type': 'USER', 'id': '55874022', 'displayNa...",closed,"[{'type': 'USER', 'id': '55874022'}]",1,2023-01-29 23:37:13.0,


In [11]:
#| export
def generate_body_update_group_membership(group_id: str,
                                          add_user_arr: list[str] = None,
                                          remove_user_arr: list[str] = None,
                                          add_owner_user_arr: list[str] = None,
                                          remove_owner_user_arr: list[str] = None) -> list[dict]:
    body = {"groupId": int(group_id)}
    if add_owner_user_arr:
        body.update({"addOwners": [{"type": "USER", "id": str(
            userId)} for userId in add_owner_user_arr]})

    if remove_owner_user_arr:
        body.update({"removeOwners": [{"type": "USER", "id": str(
            userId)} for userId in remove_owner_user_arr]})

    if remove_user_arr:
        body.update({"removeMembers": [
                    {"type": "USER", "id": str(userId)} for userId in remove_user_arr]})
    if add_user_arr:
        body.update(
            {"addMembers": [{"type": "USER", "id": str(userId)} for userId in add_user_arr]})

    return [body]

In [12]:
#| export
async def update_group_membership(auth: dmda.DomoAuth,
                                  body: dict, # first need to create body with generate_body_update_group_membership() 
                                  log_results: bool = False, 
                                  debug_api: bool = False
                                  ) -> rgd.ResponseGetData:
    # body = [{
    #     "groupId":"GROUP_ID",
    #     "removeMembers": [{"type":"USER","id":"USER_ID"}],
    #     "addMembers"   : [{"type":"USER","id":"USER_ID"}]
    # }]

    url = f'https://{auth.domo_instance}.domo.com/api/content/v2/groups/access'

    if debug_api:
        print(url, body)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method='PUT',
        body=body,
        debug_api=debug_api
    )

    return res

#### Sample implementation of generate_body_update_group_membership and update_group_membership

In [13]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community", 
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)
user = os.environ["DOMO_DOJO_USER_ID"]

# generate the body for update_group_membership-- this example adds a user
body = generate_body_update_group_membership(group_id='1259653287', add_user_arr=[user])
res = await update_group_membership(auth=token_auth, body=body)
print(res)

ResponseGetData(status=200, response='', is_success=True)


In [14]:
# #| export
# async def get_group_by_id(auth:DomoFullAuth, group_id:id, debug:bool = False) -> ResponseGetData:
#     url = f'https://{auth.domo_instance}.domo.com/api/content/v2/groups/{group_id}'

#     if debug:
#         print(url)

#     res = await gd(
#         auth=auth,
#         url=url,
#         method='GET',
#         debug=debug
#     )

#     return res

In [15]:
#| hide
import nbdev
nbdev.nbdev_export()
